In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import urllib
from pathlib import Path
from sqlalchemy import create_engine
import pyodbc

import requests
import time

In [ ]:
# Start off by creating the data information for the files

today_date = dt.date(2021, 10, 6)

today_day = today_date.strftime("%d")
today_month = today_date.strftime("%m")
today_year = today_date.strftime("%Y")

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.min_rows = 100

In [ ]:
# https://aca-prod.accela.com/MIMM/Cap/CapHome.aspx?module=Licenses&TabName=Licenses
# and
# https://aca-prod.accela.com/MIMM/Cap/CapHome.aspx?module=Adult_Use&TabName=Adult_Use&TabList=Home|0|Licenses|1|Adult_Use|2|Enforcement|3|RegistryCards|4|CurrentTabIndex|2

In [ ]:
# This creates the connection to the California Cannabis files in SQL

params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=mi_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

### Read in the Data

In [ ]:
license_info_df = pd.read_csv(f'licenses/RecordList{today_year}{today_month}{today_day}.csv')

In [ ]:
license_info_df

In [ ]:
license_info_df['Record Type'].value_counts()

### Rename Columns

In [ ]:
renamed_df = license_info_df.rename(columns={'Record Number':'license_number', 'Record Type':'license_description', 'Licensee Name':'name_legal', 'Address':'contact_address', 'Expiration Date':'expiration_date', 'Status':'license_status'})

In [ ]:
renamed_df.dtypes

In [ ]:
renamed_df['license_number'] = renamed_df['license_number'].astype(str)
renamed_df['license_description'] = renamed_df['license_description'].astype(str)
renamed_df['name_legal'] = renamed_df['name_legal'].astype(str)
renamed_df['expiration_date'] = pd.to_datetime(renamed_df['expiration_date'])

In [ ]:
renamed_df = renamed_df[['license_number', 'license_description', 'name_legal', 'contact_address', 'expiration_date', 'license_status']]

In [ ]:
renamed_df = renamed_df.sort_values(by='name_legal').reset_index().drop(columns=['index'])

In [ ]:
renamed_df['license_status'].value_counts()

In [ ]:
renamed_df

### Add Data from SQL

In [ ]:
old_mi_roll = pd.read_sql('mi_roll', con=engine_con)

In [ ]:
df_with_sql = renamed_df.merge(old_mi_roll, how='outer')

In [ ]:
df_with_sql = df_with_sql[['license_number', 'license_description', 'name_legal', 'roll_up_id', 'company_roll_up', 'contact_address', 'expiration_date', 'license_status']]

In [ ]:
df_with_sql.head()

In [ ]:
renamed_df.shape

In [ ]:
df_with_sql.shape

## Move to CSV to Add Roll Up Info

In [ ]:
df_with_sql.to_csv(f'edited_files/df_to_clean_{today_year}_{today_month}_{today_day}.csv', index=False)

### And Read it Back In and Add Roll Up Ids

In [ ]:
if sum(pd.read_csv(f'edited_files/df_to_clean_{today_year}_{today_month}_{today_day}.csv')['company_roll_up'].isnull()) == 0:
    mi_with_company_roll = pd.read_csv(f'edited_files/df_to_clean_{today_year}_{today_month}_{today_day}.csv')
else:
    print(error)

In [ ]:
# This just cleans out the NAs
mi_with_company_roll = mi_with_company_roll.fillna('')

In [ ]:
mi_with_company_roll.head()

In [ ]:
# mi_with_company_roll = mi_with_company_roll.drop(columns={'company_roll_up', 'roll_up_id'}).merge(df_with_sql[['license_number', 'roll_up_id', 'company_roll_up']])

In [ ]:
mi_roll_null = mi_with_company_roll[mi_with_company_roll['roll_up_id'] == '']
mi_roll_not_null = mi_with_company_roll[mi_with_company_roll['roll_up_id'] != '']

In [ ]:
mi_roll_null = mi_roll_null.reset_index().drop(columns='index')

In [ ]:
# This creates the lists for adding the roll_up_ids
# This will need changing when running for an update

comp_roll_list = list(mi_roll_null['company_roll_up'].unique())
roll_up_list = []

max_roll = int(sorted(mi_roll_not_null['roll_up_id'].unique())[-1][3:])

for roll in comp_roll_list:
    max_roll += 1
    new_roll = str(max_roll)
    while len(new_roll) < 5:
        new_roll = '0' + new_roll
    new_roll = 'MI-' + new_roll
    roll_up_list.append(new_roll)

In [ ]:
comp_roll_list

In [ ]:
# This adds the roll_up_ids
for n in range(len(mi_roll_null)):
    list_loc = comp_roll_list.index(mi_roll_null['company_roll_up'][n])
    mi_roll_null['roll_up_id'][n] = roll_up_list[list_loc]

In [ ]:
mi_roll_null.head()

In [ ]:
# And now we combined the two dataframes
mi_with_roll = mi_roll_not_null.append(mi_roll_null).sort_values(by='name_legal')

In [ ]:
mi_with_roll

In [ ]:
mi_with_roll[mi_with_roll['roll_up_id'] == '']

#### Clean the Contact Data

In [ ]:
# # This cleans out the emails
# for n in range(len(mi_with_roll)):
#     if '@' in (mi_with_company_roll['contact_email'][n]):
#         pass
#     else:
#         mi_with_company_roll['contact_email'][n] = ''

In [ ]:
# # This cleans out the phone numbers
# for n in range(len(mi_with_roll)):
#     mi_with_company_roll['contact_phone'][n] = mi_with_company_roll['contact_phone'][n].replace('-','').replace('.','').replace("(",'').replace(")",'').replace("\n",'').replace(" ",'')
#     if mi_with_company_roll['contact_phone'][n] == 'https://wwwinstagramcom/grownrogue/':
#         mi_with_company_roll['contact_phone'][n] = ''
#     if mi_with_company_roll['contact_phone'][n] == '1833VIRIDIS':
#         mi_with_company_roll['contact_phone'][n] = '18338474347'

In [ ]:
# # This cleans out the websites
# for n in range(len(mi_with_roll)):
#     if 'facebook.com' in mi_with_company_roll['contact_website'][n]:
#         mi_with_company_roll['contact_website'][n] = ''
#     if mi_with_company_roll['contact_website'][n] == 'https://www.google.com/search?q=BLUMFIELD+SECURE+TRANSPORT+LLC&rlz=1C5CHFA_enUS906US907&oq=BLUMFIELD+SECURE+TRANSPORT+LLC&aqs=chrome..69i57.121j0j4&sourceid=chrome&ie=UTF-8':
#         mi_with_company_roll['contact_website'][n] = 'https://blumfieldsecuretransport.com/'

In [ ]:
# mi_with_company_roll['contact_phone'].unique()

In [ ]:
mi_with_roll[mi_with_roll['license_number'] == 'AU-ER-001875']

### Compare to SQL

In [ ]:
# This seperates the license numbers not already in SQL
old_sql_main = pd.read_sql('mi_main', con=engine_con)

old_lic_nums = old_sql_main['license_number'].unique()
lic_nums = mi_with_roll['license_number'].unique()
new_lic_nums = []

for lic in lic_nums:
    if lic not in old_lic_nums:
        new_lic_nums.append(lic)

In [ ]:
# And now we make the dataframe out of the new licenses
new_mi_df = pd.DataFrame(new_lic_nums, columns={'license_number'})
new_mi_df = new_mi_df.merge(mi_with_roll, on='license_number')

In [ ]:
new_mi_df['is_current'] = 1
new_mi_df['date_uploaded'] = today_date

In [ ]:
new_mi_df[new_mi_df['roll_up_id'] == '']

## Seperate for SQL

In [ ]:
mi_roll = new_mi_df[['license_number', 'roll_up_id', 'company_roll_up']]

In [ ]:
mi_main = new_mi_df[['license_number', 'name_legal', 'license_description', 'license_status', 'is_current', 'expiration_date', 'date_uploaded']]

In [ ]:
mi_contact = new_mi_df[['license_number', 'contact_address']]
#                         'contact_email', 'contact_phone', 'contact_website']]

In [ ]:
mi_roll.to_sql('mi_roll', con=engine_con, index=False, if_exists='append')

In [ ]:
mi_main.to_sql('mi_main', con=engine_con, index=False, if_exists='append')

In [ ]:
mi_contact.to_sql('mi_contact', con=engine_con, index=False, if_exists='append')

In [ ]:
mi_main

### Create the to_alter Table

In [ ]:
current_mi_main = mi_with_roll[mi_with_roll['name_legal'] != ''][['license_number', 'name_legal', 'license_description', 'license_status', 'expiration_date']]

In [ ]:
current_mi_main = current_mi_main.sort_index()

In [ ]:
list_current = list(current_mi_main['license_number'].unique())

In [ ]:
not_current_df = pd.read_sql('mi_main', con=engine_con)

In [ ]:
not_current_list = []

for lic in not_current_df['license_number']:
    if lic not in list_current:
        not_current_list.append(lic)
        
not_current_df = not_current_df[not_current_df['license_number'].isin(not_current_list)]

In [ ]:
not_current_df['is_current'] = 0

In [ ]:
up_to_date_df = current_mi_main.append(not_current_df).reset_index().drop(columns='index')
up_to_date_df['is_current'] = up_to_date_df['is_current'].fillna(1).astype(int)
up_to_date_df['expiration_date'] = pd.to_datetime(up_to_date_df['expiration_date'])
up_to_date_df['expiration_date'] = up_to_date_df['expiration_date'].dt.strftime("%m/%d/%y")

In [ ]:
old_data_df = pd.read_sql('mi_main', con=engine_con)
old_data_df['expiration_date'] = pd.to_datetime(old_data_df['expiration_date'])
old_data_df['expiration_date'] = old_data_df['expiration_date'].dt.strftime("%m/%d/%y")

In [ ]:
up_to_date_df

In [ ]:
# This code creates a list of all the rows that need changing (and specifies what column needs changing so we can double check)

for_edit_list = []

for lic in up_to_date_df['license_number'].unique():
    
    if list(up_to_date_df[up_to_date_df['license_number'] == lic]['license_status'])[0] != list(old_data_df[old_data_df['license_number'] == lic]['license_status'])[0]:
        print(lic, 'license_status')
        for_edit_list.append(lic)
        
    elif list(up_to_date_df[up_to_date_df['license_number'] == lic]['license_description'])[0] != list(old_data_df[old_data_df['license_number'] == lic]['license_description'])[0]:
        print(lic, 'license_description')
        for_edit_list.append(lic)
        
    elif list(up_to_date_df[up_to_date_df['license_number'] == lic]['expiration_date'])[0] != list(old_data_df[old_data_df['license_number'] == lic]['expiration_date'])[0]:
        print(lic, 'expiration_date')
        for_edit_list.append(lic)
        
    elif list(up_to_date_df[up_to_date_df['license_number'] == lic]['is_current'])[0] != list(old_data_df[old_data_df['license_number'] == lic]['is_current'])[0]:
        print(lic, 'is_current')
        for_edit_list.append(lic)
        

In [ ]:
len(for_edit_list)

In [ ]:
# This creates the dataframe to send to SQL
to_alter_df_all = up_to_date_df[up_to_date_df['license_number'].isin(for_edit_list)]
# to_alter_df_all['expiration_date'] = to_alter_df_all['expiration_date'].astype(str)

In [ ]:
# This sends the newly created dataframe to SQL
to_alter_df_all.to_sql('mi_main_alter', con=engine_con, index=False, if_exists='replace')

In [ ]:
# This code runs the query noramally run in SQL to update mi_main using ca_main_alter

mi_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=mi_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

mi_cursor = mi_conn.cursor()

mi_cursor.execute('SELECT * FROM mi_cannabis.dbo.mi_main')

mi_cursor.execute('''
Update t1
SET t1.[license_status] = t2.[license_status],
    t1.[license_description] = t2.[license_description],
    t1.[expiration_date] = t2.[expiration_date],
    t1.[is_current] = t2.[is_current]
    
FROM [mi_cannabis].[dbo].[mi_main] as t1
INNER JOIN [mi_cannabis].[dbo].[mi_main_alter] as t2
ON t1.[license_number] = t2.[license_number];
                  ''')

mi_conn.commit()

### Move to all_main

In [ ]:
# This creates the connection to the all_main file in SQL

all_cannabis_params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=all_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
all_cannabis_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(all_cannabis_params))

In [ ]:
# This reads in the all_main data and narrows it down to the relevent data
all_main_sql = pd.read_sql('all_main', con=all_cannabis_con)
mi_main_sql = all_main_sql[all_main_sql['us_state'] == 'MI']
mi_main_sql = mi_main_sql.reset_index().drop(columns='index')

In [ ]:
# This re-reads in and merges the ca_annabis data in SQL
updated_mi_main = pd.read_sql('mi_main', con=engine_con)
updated_mi_roll = pd.read_sql('mi_roll', con=engine_con)
updated_mi_cannabis = updated_mi_main.merge(updated_mi_roll, on='license_number')

In [ ]:
mi_main_sql

In [ ]:
updated_mi_cannabis

In [ ]:
updated_mi_cannabis['us_state'] = 'MI'

In [ ]:
updated_mi_cannabis['license_category'] = ''

for n in range(len(updated_mi_cannabis)):
    if updated_mi_cannabis['license_description'].iloc[n] in ['Class A Grower – License', 'Class B Grower – License', 'Class C Grower – License',
                                                             'Class A Marihuana Grower - License', 'Class B Marihuana Grower - License', 'Class C Marihuana Grower - License', 
                                                             'Excess Marihuana Grower - License', 'Grower License – Class C – 1500 Plants – License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Cultivator'
        
    elif updated_mi_cannabis['license_description'].iloc[n] in ['Marihuana Retailer - License', 'Provisioning Center - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Retailer'
        
    elif updated_mi_cannabis['license_description'].iloc[n] in ['Processor - License', 'Marihuana Processor - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Manufacturer'
        
    elif updated_mi_cannabis['license_description'].iloc[n] in ['Secure Transporter - License', 'Marihuana Secure Transporter - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Distributor'
    
    elif updated_mi_cannabis['license_description'].iloc[n] in ['Safety Compliance - License', 'Marihuana Safety Compliance Facility - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Laboratory'
    
    elif updated_mi_cannabis['license_description'].iloc[n] in ['Marihuana Microbusiness - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Multiple'

    elif updated_mi_cannabis['license_description'].iloc[n] in ['Entity Prequalification', 'Individual Prequalification',
                                                                'Adult-Use Entity Registration - License', 'Sole Proprietor Registration - License',
                                                                'Marihuana Event Organizer - License', 'Temporary Marihuana Event - License']:
        updated_mi_cannabis['license_category'].iloc[n] = 'Other'

In [ ]:
updated_mi_cannabis[updated_mi_cannabis['license_category'] == '']['license_description'].value_counts()

In [ ]:
updated_mi_cannabis[updated_mi_cannabis['license_description'] == 'Adult-Use Entity Registration - License']

In [ ]:
not_in_all_main = []

for lic in list(updated_mi_cannabis['license_number'].unique()):
    if lic not in list(mi_main_sql['license_number'].unique()):
        not_in_all_main.append(lic)
        
new_all_main = updated_mi_cannabis[updated_mi_cannabis['license_number'].isin(not_in_all_main)]

In [ ]:
new_all_main = new_all_main[['license_number', 'roll_up_id', 'name_legal', 'license_status', 'is_current', 'license_description', 'us_state', 'license_category', 'expiration_date']]

In [ ]:
new_all_main.head()

In [ ]:
new_all_main.dtypes

In [ ]:
new_all_main[new_all_main['is_current'].isnull()]

In [ ]:
new_all_main.shape

In [ ]:
# Because we renamed the all_main columns, just need to touch this up fast
new_all_main = new_all_main.rename(columns={'roll_up_id':'rollup_id', 'name_legal':'legal_name'})

In [ ]:
new_all_main.to_sql('all_main', con=all_cannabis_con, if_exists='append', index=False)

### main_alter for all_main

In [ ]:
mi_main_sql['expiration_date'] = pd.to_datetime(mi_main_sql['expiration_date']).dt.strftime("%m/%d/%y")
updated_mi_cannabis['expiration_date'] = pd.to_datetime(updated_mi_cannabis['expiration_date']).dt.strftime("%m/%d/%y")

In [ ]:
mi_all_main_for_alter_list = []

for lic in list(mi_main_sql['license_number'].unique()):
    if list(mi_main_sql[mi_main_sql['license_number'] == lic]['is_current'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['is_current'])[0]:
        print(lic, 'is_current')
        mi_all_main_for_alter_list.append(lic)
    elif list(mi_main_sql[mi_main_sql['license_number'] == lic]['expiration_date'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['expiration_date'])[0]:
        print(lic, 'expiration_date')
        mi_all_main_for_alter_list.append(lic)
    elif list(mi_main_sql[mi_main_sql['license_number'] == lic]['rollup_id'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['roll_up_id'])[0]:
        print(lic, 'roll_up_id')
        mi_all_main_for_alter_list.append(lic)
    elif list(mi_main_sql[mi_main_sql['license_number'] == lic]['license_status'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['license_status'])[0]:
        print(lic, 'license_status')
        mi_all_main_for_alter_list.append(lic)
    elif list(mi_main_sql[mi_main_sql['license_number'] == lic]['license_description'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['license_description'])[0]:
        print(lic, 'license_description')
        mi_all_main_for_alter_list.append(lic)
    elif list(mi_main_sql[mi_main_sql['license_number'] == lic]['license_category'])[0] != list(updated_mi_cannabis[updated_mi_cannabis['license_number'] == lic]['license_category'])[0]:
        print(lic, 'license_category')
        mi_all_main_for_alter_list.append(lic)
        
mi_all_main_for_alter_df = updated_mi_cannabis[updated_mi_cannabis['license_number'].isin(mi_all_main_for_alter_list)]

In [ ]:
mi_all_main_for_alter_df

In [ ]:
mi_all_main_for_alter_df.shape

In [ ]:
mi_all_main_for_alter_df = mi_all_main_for_alter_df.rename(columns={'name_legal':'legal_name', 'roll_up_id':'rollup_id'})

In [ ]:
mi_all_main_for_alter_df.to_sql('mi_to_alter', con=all_cannabis_con, if_exists='replace' ,index=False)

In [ ]:
# And this code runs the query to update the all_main table with the mi_to_alter table

all_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=all_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

all_cursor = all_conn.cursor()

all_cursor.execute('SELECT * FROM all_cannabis.dbo.all_main')

all_cursor.execute('''
Update t1
SET t1.[license_status] = t2.[license_status],
    t1.[license_description] = t2.[license_description],
    t1.[license_category] = t2.[license_category],
    t1.[is_current] = t2.[is_current],
    t1.[rollup_id] = t2.[rollup_id],
    t1.[expiration_date] = t2.[expiration_date]
FROM [all_cannabis].[dbo].[all_main] as t1
INNER JOIN [all_cannabis].[dbo].[mi_to_alter] as t2
ON t1.[license_number] = t2.[license_number];
''')

all_conn.commit()

### Bespoke/Metabase

In [ ]:
new_main = new_all_main.append(mi_all_main_for_alter_df).reset_index().drop(columns='index')

In [ ]:
new_main['expiration_date'] = new_main['expiration_date'].astype(str)

In [ ]:
new_main.shape

In [ ]:
num_of_lic_sent = 0

us_state = 'MI'

while len(new_main) - num_of_lic_sent >= 10:

    n = num_of_lic_sent
    
    if new_main['is_current'].iloc[n] == 1:
        is_current_1 = True
    else:
        is_current_1 = False
            
    if new_main['is_current'].iloc[n+1] == 1:
        is_current_2 = True
    else:
        is_current_2 = False
          
    if new_main['is_current'].iloc[n+2] == 1:
        is_current_3 = True
    else:
        is_current_3 = False
            
    if new_main['is_current'].iloc[n+3] == 1:
        is_current_4 = True
    else:
        is_current_4 = False
            
    if new_main['is_current'].iloc[n+4] == 1:
        is_current_5 = True
    else:
        is_current_5 = False
            
    if new_main['is_current'].iloc[n+5] == 1:
        is_current_6 = True
    else:
        is_current_6 = False
            
    if new_main['is_current'].iloc[n+6] == 1:
        is_current_7 = True
    else:
        is_current_7 = False
            
    if new_main['is_current'].iloc[n+7] == 1:
        is_current_8 = True
    else:
        is_current_8 = False
            
    if new_main['is_current'].iloc[n+8] == 1:
        is_current_9 = True
    else:
        is_current_9 = False
            
    if new_main['is_current'].iloc[n+9] == 1:
        is_current_10 = True
    else:
        is_current_10 = False
            

    requests.post(
#             'https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            json= {
          'company_licenses': [
              
              {
            'license_number': new_main['license_number'][n],
            'rollup_id': new_main['rollup_id'][n],
            'legal_name': new_main['legal_name'][n],
            'license_status': new_main['license_status'][n],
            'is_current': is_current_1,
            'license_description': new_main['license_description'][n],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n],
            'license_category': new_main['license_category'][n]
            },
            
              
              {
            'license_number': new_main['license_number'][n+1],
            'rollup_id': new_main['rollup_id'][n+1],
            'legal_name': new_main['legal_name'][n+1],
            'license_status': new_main['license_status'][n+1],
            'is_current': is_current_2,
            'license_description': new_main['license_description'][n+1],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+1],
            'license_category': new_main['license_category'][n+1]
            },
              
              
              {
            'license_number': new_main['license_number'][n+2],
            'rollup_id': new_main['rollup_id'][n+2],
            'legal_name': new_main['legal_name'][n+2],
            'license_status': new_main['license_status'][n+2],
            'is_current': is_current_3,
            'license_description': new_main['license_description'][n+2],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+2],
            'license_category': new_main['license_category'][n+2]
            },

              
              {
            'license_number': new_main['license_number'][n+3],
            'rollup_id': new_main['rollup_id'][n+3],
            'legal_name': new_main['legal_name'][n+3],
            'license_status': new_main['license_status'][n+3],
            'is_current': is_current_4,
            'license_description': new_main['license_description'][n+3],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+3],
            'license_category': new_main['license_category'][n+3]
            },
              
              
              {
            'license_number': new_main['license_number'][n+4],
            'rollup_id': new_main['rollup_id'][n+4],
            'legal_name': new_main['legal_name'][n+4],
            'license_status': new_main['license_status'][n+4],
            'is_current': is_current_5,
            'license_description': new_main['license_description'][n+4],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+4],
            'license_category': new_main['license_category'][n+4]
            },
              
              
              {
            'license_number': new_main['license_number'][n+5],
            'rollup_id': new_main['rollup_id'][n+5],
            'legal_name': new_main['legal_name'][n+5],
            'license_status': new_main['license_status'][n+5],
            'is_current': is_current_6,
            'license_description': new_main['license_description'][n+5],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+5],
            'license_category': new_main['license_category'][n+5]
            },

              
              {
            'license_number': new_main['license_number'][n+6],
            'rollup_id': new_main['rollup_id'][n+6],
            'legal_name': new_main['legal_name'][n+6],
            'license_status': new_main['license_status'][n+6],
            'is_current': is_current_7,
            'license_description': new_main['license_description'][n+6],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+6],
            'license_category': new_main['license_category'][n+6]
            },
              
              
              {
            'license_number': new_main['license_number'][n+7],
            'rollup_id': new_main['rollup_id'][n+7],
            'legal_name': new_main['legal_name'][n+7],
            'license_status': new_main['license_status'][n+7],
            'is_current': is_current_8,
            'license_description': new_main['license_description'][n+7],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+7],
            'license_category': new_main['license_category'][n+7]
            },
              
              
              {
            'license_number': new_main['license_number'][n+8],
            'rollup_id': new_main['rollup_id'][n+8],
            'legal_name': new_main['legal_name'][n+8],
            'license_status': new_main['license_status'][n+8],
            'is_current': is_current_9,
            'license_description': new_main['license_description'][n+8],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+8],
            'license_category': new_main['license_category'][n+8]
            },

              
              {
            'license_number': new_main['license_number'][n+9],
            'rollup_id': new_main['rollup_id'][n+9],
            'legal_name': new_main['legal_name'][n+9],
            'license_status': new_main['license_status'][n+9],
            'is_current': is_current_10,
            'license_description': new_main['license_description'][n+9],
            'us_state': us_state,
            'expiration_date': new_main['expiration_date'][n+9],
            'license_category': new_main['license_category'][n+9]
            },

              
          ],
        },
            headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
    )
        
    num_of_lic_sent += 10
        
    time.sleep(5)
    
    print(num_of_lic_sent)
    
    
    
        
if len(new_main) - num_of_lic_sent < 10:
            
    for n in range(num_of_lic_sent, len(new_main)):
    
        if new_main['is_current'].iloc[n] == 1:
            is_current = True
        else:
            is_current = False
                    
                    
        requests.post(
                'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
                json= {
                'company_licenses': [
              
                                      {
                                        'license_number': new_main['license_number'].iloc[n],
                                        'rollup_id': new_main['rollup_id'].iloc[n],
                                        'legal_name': new_main['legal_name'].iloc[n],
                                        'license_status': new_main['license_status'].iloc[n],
                                        'is_current': is_current,
                                        'license_description': new_main['license_description'].iloc[n],
                                        'us_state': us_state,
                                        'expiration_date': new_main['expiration_date'].iloc[n],
                                        'license_category': new_main['license_category'].iloc[n]
                                    },
                                  ],
                                },
                                    headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
                            )
        
        num_of_lic_sent += 1
        
        print(num_of_lic_sent)

In [ ]:
metabase_json = pd.read_json('query_result_2021-09-30T15_03_41.105Z.json')

In [ ]:
for lic in new_main['license_number'].unique():
    if lic not in metabase_json['License Number'].unique():
        print(lic)

In [ ]:
new_main[new_main['license_number'] == 'AU-SP-000172']